# Simple example on use of Julia JuMP for a LP problem.

The company Paint Deals produces two colors of paint, blue and black.
- Blue paint is sold for US\\$10 per liter, while black paint is sold for US\\$15
per liter.
- The company owns a process plant which can produce one color paint at
a time.
- However, blue paint is produced at a rate of 40 liters per hour, while the
production rate for black paint is 30 liters per hour.
- Besides, the marketing department estimates that at most 860 liters of
black paint and 1000 liters of blue paint can be sold in the market.
- During a week, the plant can operate for 40 hours and the paint can be
stored for the following week.
- Determine how many liters of each paint should be produced to maximize
week revenue.

This problem can be written as a linear program:
\begin{align}
\max \quad & 10 \cdot \text{BluePaint} + 15 \cdot \text{BlackPaint} \\
 \text{subject to} \quad & \frac{1}{40} \cdot \text{BluePaint} + \frac{1}{30} \cdot \text{BlackPaint} \leq 40 \\
 & 0 \leq \text{BluePaint} \leq 860 \\
 & 0 \leq \text{BlackPaint} \leq 1000 
\end{align}

We first need to tell Julia that we are using JuMP (has to be installed beforehand):

In [6]:
using JuMP

Then we need to include an appropriate solver for the optimization problem. In this case the solver is GLPK (must also be installed beforehand).

In [7]:
using GLPK

In JuMP, we define our optimization problem as an (optimization) model, using the model object. The model object is a container for variables, constraints, objective, solver options, etc. for the optimization problem. When we create the model object, we can tell the constructor which optimizer we will use:

In [8]:
model = Model(GLPK.Optimizer);

Then we need to define the optimization variables. We use `@variable(name of the model object, variable name and bound, variable type)` to do this. The bound can be a lower bound, an upper bound, or both. If no variable type is specified, then the default is real.

In [9]:
@variable(model, 0 <= BluePaint <= 860)
@variable(model, 0 <= BlackPaint <= 1000);

We specify constraints using `@constraint(name of the model object, constraint)`. In this case, we have only one constraint other than the variable bounds:

In [10]:
@constraint(model, (1/40)*BluePaint + (1/30)*BlackPaint <= 40);

In a similar manner, we specify the objective using `@objective(name of the model object, Min/Max, function to be optimized)`:

In [11]:
@objective(model, Max, 10*BluePaint + 15*BlackPaint);

We are now ready to solve the optimization problem, using the `optimize` function. Note that the solution will be stored within the Model object and the variables.

In [26]:
optimize!(model)

We can print the value of the variables at the solution, and the objective function value.

In [27]:
@show value(BluePaint);
@show value(BlackPaint);
@show objective_value(model);

value(BluePaint) = 266.6666666666666
value(BlackPaint) = 1000.0
objective_value(model) = 17666.666666666664


Often, we use vector variables. We can specify the optimization problem using vector variables in the following way:

In [25]:
model2 = Model(GLPK.Optimizer)
@variable(model2, (0; 0) <= x[1:2] <= (860; 1000) )

@constraint(model2, (1/40)*x[1] + (1/30)*x[2] <= 40)

@objective(model2, Max, 10*x[1] + 15*x[2])

optimize!(model2)

@show value.(x);
@show objective_value(model2);

value.(x) = [266.6666666666666, 1000.0]
objective_value(model2) = 17666.666666666664
